## Install Dependencies

In [ ]:
!pip3 install python-terrier

In [13]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
from tqdm import tqdm
import gzip
import json

In [14]:
ensure_pyterrier_is_loaded()
tira = Client()

## Load Dataset and Index

In [15]:
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)
#documents = pt_dataset.get??()
#print(documents).head(3)
print(dir(pt_dataset))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_configure', '_describe_component', '_irds_id', '_irds_ref', 'get_corpus', 'get_corpus_iter', 'get_corpus_lang', 'get_index', 'get_qrels', 'get_results', 'get_topics', 'get_topics_lang', 'get_topicsqrels', 'info_url', 'irds_ref']


## Document Expansion with DocT5Query
> https://github.com/castorini/docTTTTTquery
>
> https://github.com/tira-io/teaching-ir-with-shared-tasks/blob/main/tutorials/tutorial-doc-t5-query.ipynb

In [17]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
# bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', pt_dataset)

In [18]:
def doc_t5_query(dataset):
    docs = tira.get_run_output('ir-benchmarks/seanmacavaney/DocT5Query', dataset) + '/documents.jsonl.gz'
    with gzip.open(docs, 'rt') as f:
        for l in tqdm(f):
            l = json.loads(l)
            l['text'] = l['querygen']
            l['docno'] = l['doc_id']
            del l['doc_id']
            del l['querygen']
            yield l

def doc_t5_query_index(dataset):
    indexer = pt.IterDictIndexer("/tmp/index2", overwrite=True, meta={'docno': 100, 'text': 20480})
    index_ref = indexer.index(doc_t5_query(dataset))
    return pt.IndexFactory.of(index_ref)

In [20]:
index = pt.IndexFactory.of('/tmp/index2')

JavaException: JVM exception occurred: No IndexLoaders were supported for indexref /tmp/index; It may be your ref has the wrong location. Alternatively, Terrier is misconfigured - did you import the correct package to deal with this indexref? java.lang.UnsupportedOperationException

In [ ]:
docs_retrieved_by_bm25 = {}

bm25_result = bm25(pt_dataset.get_topics('title'))

for _, i in tqdm(bm25_result.iterrows()):
    qid, docno = str(i['qid']), str(i['docno'])

    if qid not in docs_retrieved_by_bm25:
        docs_retrieved_by_bm25[qid] = set()
    
    docs_retrieved_by_bm25[qid].add(docno)

In [ ]:
omit_already_retrieved_docs = lambda i: i[i.apply(lambda j: str(j['docno']) not in docs_retrieved_by_bm25[str(j['qid'])], axis=1)]
omit_already_retrieved_docs = pt.apply.generic(omit_already_retrieved_docs)

In [ ]:
bm25_doct5query = pt.BatchRetrieve(index, wmodel="BM25")

bm25_doct5query_new = bm25_doct5query >> omit_already_retrieved_docs


## Stopword removal
> https://github.com/tira-io/teaching-ir-with-shared-tasks/blob/main/tutorials/tutorial-stopword-lists.ipynb

In [ ]:
# does not work yet because I have no idea how to access the documents in the dataset
"""def create_index(documents, stopwords):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords=stopwords)
    index_ref = indexer.index(documents)
    return pt.IndexFactory.of(index_ref)

index = create_index(documents, stopwords=['the','he','she','it','they','them','their'])"""

In [ ]:
# bm25 = pt.BatchRetrieve(index, wmodel="BM25")

## Bo1 Query Expansion
> https://github.com/tira-io/teaching-ir-with-shared-tasks/blob/main/tutorials/tutorial-query-expansion.ipynb

In [ ]:
bo1_expansion = bm25_doct5query_new >> pt.rewrite.Bo1QueryExpansion(index)

In [ ]:
# build final pipeline for retrieval
bm25_bo1 = bo1_expansion >> bm25

In [23]:
bm25_bo1.search('query expansion').head(10)

,qid,docid,docno,rank,score,query_0,query
0,1,84088,2003.sigirconf_conference-2003.30,0,18.226092,query expansion,applypipeline:off queri^1.154975233 expans^1.4...
1,1,82474,1998.sigirconf_conference-98.17,1,17.993732,query expansion,applypipeline:off queri^1.154975233 expans^1.4...
2,1,96536,2001.cikm_conference-2001.24,2,17.926357,query expansion,applypipeline:off queri^1.154975233 expans^1.4...
3,1,74419,2005.ntcir_workshop-2005.58,3,17.115286,query expansion,applypipeline:off queri^1.154975233 expans^1.4...
4,1,101016,2004.spire_conference-2004.4,4,17.019983,query expansion,applypipeline:off queri^1.154975233 expans^1.4...
5,1,124131,2007.ipm_journal-ir0volumeA43A4.1,5,17.007919,query expansion,applypipeline:off queri^1.154975233 expans^1.4...
6,1,126479,2001.tois_journal-ir0volumeA19A1.0,6,16.980183,query expansion,applypipeline:off queri^1.154975233 expans^1.4...
7,1,69797,2021.ccl-1.84,7,16.809470,query expansion,applypipeline:off queri^1.154975233 expans^1.4...
8,1,38272,C10-2007,8,16.790618,query expansion,applypipeline:off queri^1.154975233 expans^1.4...
9,1,108448,2002.wwwconf_conference-2002.32,9,16.784934,query expansion,applypipeline:off queri^1.154975233 expans^1.4...


## Evaluation

In [21]:
## actually no clue what this means
pt.Experiment(
    retr_systems=[bm25_bo1,bm25_doct5query_new,bm25],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['BM25-Bo1-docT5query','BM25-docT5-query','BM25'],
    eval_metrics=['recall_1000','recip_rank','map']
)

NameError: name 'bm25_doct5query_new' is not defined